In [58]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.model_selection import train_test_split
eps=np.finfo(float).eps
from binarytree import tree,Node
from sklearn.metrics import classification_report, confusion_matrix ,accuracy_score
from sklearn.tree import DecisionTreeClassifier
from operator import itemgetter
import copy

In [59]:
df=pd.read_csv("train.csv")

In [60]:
X =df.drop(['left','last_evaluation','satisfaction_level','average_montly_hours','time_spend_company','number_project','salary','sales'],axis=1)
X = pd.concat([X,pd.get_dummies(df['sales'], prefix='sales')],axis=1)
X = pd.concat([X,pd.get_dummies(df['salary'], prefix='salary')],axis=1)
y=df['left']

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
orignalX=copy.deepcopy(X_train)
orignalY=copy.deepcopy(y_train)
testX=copy.deepcopy(X_test)
testY=copy.deepcopy(y_test)

In [62]:
df=pd.concat([X_train,y_train],axis=1)
# df

In [63]:
def bestAttribute(df):
    if df.empty==True:
        return
#     infoGain=[]
    max_attr=""
    max1=0
    entropy1=findParentEntropy(df)
    for key in df.keys()[:-1]:
#         print("dwfa  ",key)
        entropy2=findRelativeEntropy(df,key)
        if (entropy1-entropy2)>max1:
            max1=entropy1-entropy2
            max_attr=key
        
    return max_attr
    

In [64]:
def findRelativeEntropy(df,attribute):
    if df.empty==True:
        return
#  
    classCol='left'
    resultValues=df['left'].unique() 
    attributeNames=df[attribute].unique()
    entropy2 = 0
    for attr in attributeNames:
        entropy = 0
        for value in resultValues:
            #temp&hot with yes
            num = len(df[attribute][df[attribute]==attr][df[classCol]==value])
            #total hot
            den = len(df[attribute][df[attribute]==attr])
            fraction = num/(den+eps)
            entropy += -fraction*np.log2(fraction+eps)
        #entropy attribute/total*(entropy attribute_values)
        fraction2 = den/len(df)  
        entropy2 += -fraction2*entropy 
    return abs(entropy2)


In [65]:
def findParentEntropy(df):
    if df.empty==True:
        return
    classCol=df.keys()[-1]
    #yeh sirf header return karega
    resultValues=df[classCol].unique()
    #all ouput values in output column
    entropy=0
    for value in resultValues:
        fraction=df[classCol].value_counts()[value]/len(df[classCol])
        entropy+=-fraction*np.log2(fraction+eps)
    return abs(entropy)

In [66]:
class node:
    def __init__(self,value,left=None,right=None):
        self.value=value
        self.left=left
        self.right=right

In [67]:
def get_subtable(df, node,value):
    if df.empty==True:
        return
    return df[df[node] == value].reset_index(drop=True)


In [68]:
def buildTree(df):
    if len(df.columns)==1:
        return
    
    clValue=df['left'].unique()
    r=0;
    counts=[]
    counts=df['left'].value_counts()
    attribute_name=bestAttribute(df)
#     attValue = np.unique(df[attribute_name])
    if attribute_name=="":
            return None
    root=node(attribute_name)
    
    if len(counts)<=1:
        if clValue[0]==0:
            root.negative=counts[0]
        else:
            root.positive=counts[0]
    else:
        root.positive=counts[1]
        root.negative=counts[0]
            
            
#         for value in attValue:
#             print(type(value))
        
        subtable0 = get_subtable(df,attribute_name,0)
        subtable0 =subtable0.drop([attribute_name],axis=1)
        subtable1 = get_subtable(df,attribute_name,1)
        subtable1 =subtable1.drop([attribute_name],axis=1)
        countleft=subtable0['left'].value_counts()
        countright=subtable1['left'].value_counts()
        if(len(countleft)<=1):
            pass
        else:
            root.left=buildTree(subtable0) #Calling the function recursively
        if(len(countright)<=1):
            pass
        else:
            root.right=buildTree(subtable1)
       
                   
    return root

In [69]:
root=buildTree(df)
# print(root.value)

In [70]:
# print(root.right.value)
# print(root.right.left)
# print(root.left.value)
# print(root.left.left.value)
# print(root.left.right.value)


In [71]:
def testing(row1,root):
    #     if root==None:
    #         return 
        #print(root.value)
        if root.left==None and root.right==None:
            if root.positive > root.negative:
                y1.append(1)
            else:
                y1.append(0)
            return
    #     for i in row1.keys():
    #         if i==root.value:
        i=root.value
        if i in row1:
            
            if row1[i]==0:
        #         print("000")
                if root.left==None:
                    if root.positive > root.negative:
                        y1.append(1)
                        return
                    else:
                        y1.append(0)
                        return
                else:
                    testing(row1,root.left)
            else:
        #         print("111")
                if root.right==None:
                    if root.positive > root.negative:
                        y1.append(1)
                        return
                    else:
                        y1.append(0)
                        return 
                else:
                    testing(row1,root.right)
        else:
            if root.left==None:
                if root.positive > root.negative:
                    y1.append(1)
                    return
                else:
                    y1.append(0)
                    return
            else:
                testing(row1,root.left)

In [72]:
# def testingData(df):
#     for index,row in df.iterrows():
#         testing(row,root)

# y1=[]       
# testingData(X_test)  
# print(len(y1))
def predict(df,root):
    for index,row in df.iterrows():
        testing(row,root)

y1=[] 
X_test
predict(X_test,root)  
print(len(y1))



2248


In [73]:
print(confusion_matrix(y_test, y1))  
print(classification_report(y_test, y1)) 

[[1706    0]
 [ 542    0]]
              precision    recall  f1-score   support

           0       0.76      1.00      0.86      1706
           1       0.00      0.00      0.00       542

   micro avg       0.76      0.76      0.76      2248
   macro avg       0.38      0.50      0.43      2248
weighted avg       0.58      0.76      0.65      2248



/home/dhawal/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [74]:
accuracy=accuracy_score(y_test, y1)*100
print(accuracy)

75.88967971530249


In [75]:
inputData=pd.read_csv("sample_test.csv")
XInput =inputData.drop(['sales','salary','last_evaluation','satisfaction_level','average_montly_hours','time_spend_company','number_project'],axis=1)
XInput = pd.concat([XInput,pd.get_dummies(inputData['sales'], prefix='sales')],axis=1)
XInput = pd.concat([XInput,pd.get_dummies(inputData['salary'], prefix='salary')],axis=1)
y1=[] 

       
predict(XInput,root)

In [76]:
print(y1)

[0, 0]


In [77]:
classifier = DecisionTreeClassifier()
classifier.fit(orignalX, orignalY)  
y_pred = classifier.predict(testX)  
print(confusion_matrix(testY, y_pred))  
print(classification_report(testY, y_pred)) 
print(accuracy_score(testY,y_pred))

[[1705    1]
 [ 541    1]]
              precision    recall  f1-score   support

           0       0.76      1.00      0.86      1706
           1       0.50      0.00      0.00       542

   micro avg       0.76      0.76      0.76      2248
   macro avg       0.63      0.50      0.43      2248
weighted avg       0.70      0.76      0.66      2248

0.7588967971530249


In [78]:
accuracy=accuracy_score(y_test, y_pred)*100
print(accuracy)

75.88967971530249
